In [1]:
from wandb.keras import WandbCallback
import wandb
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import numpy as np
from model import *
from dataloader import *
from tensorflow.keras.utils import to_categorical

In [2]:
dataset = Dataset()
dataset.load()

Loading & encoding data from disk.


100%|██████████| 500/500 [00:10<00:00, 49.61it/s]


In [3]:
sentences = dataset.getConcatenatedTracks()
beatdict = BeatDictionary(sentences)
beatdict.get_related_chords('56-0.5-0')

Similar chords with 56-0.5-0
56-0.5-1 0.983
56-0.25-1#58-0.25-1 0.968
56-0.25-0#58-0.25-1 0.953


In [4]:
x_train, y_train = dataset.getSingleTrackTrainingData(track=0)

In [5]:
x_train = np.array([beatdict.vectorize(beat.tolist()) for beat in x_train])

y_train = beatdict.beat2index(y_train)
y_train = to_categorical(y_train, num_classes=beatdict.getDictSize())
# y_train = np.array([beatdict.vectorize(beat.tolist()) for beat in y_train])

In [6]:
print(x_train.shape, y_train.shape)
print(beatdict.getDictSize())

(23174, 1, 100) (23174, 1360)
1360


In [7]:
model = LoFiLoopNet("exp0618", x_train.shape[1:], beatdict.getDictSize())
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 256)            365568    
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 512)            1574912   
_________________________________________________________________
dropout (Dropout)            (None, 1, 512)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               1

In [8]:
model.train(x_train, y_train)

wandb: Currently logged in as: tku-cilab (use `wandb login --relogin` to force relogin)


wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.


Epoch 1/50


: 

: 

In [ ]:
sequence = model.generate(dataset, beatdict)
new_midi = MidiData()
new_midi.decode(sequence, filename='workflow_test4.mid')

# Old

In [ ]:
sequence

In [ ]:
print(dataset.pitch_range_list)

In [ ]:
model = LoFiLoopNet()
model.train(x_train, y_train)
model.plot()

In [ ]:
model = LoFiLoopNet()
model.load_last_weight()

In [ ]:
model.generate(dataset)